In [ ]:
!git clone https://github.com/abhishict/TextMiningProject.git

Cloning into 'TextMiningProject'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 16 (delta 5), reused 5 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (16/16), 1.93 MiB | 4.24 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [ ]:
!pip install pyLDAvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 905.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 7.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1


In [ ]:
import pandas as pd
df = pd.read_csv("/content/TextMiningProject/data/cleaned_dataset.csv")
df.head()

,Label,Description,Cleaned_Description,Article_Length
0,Environmental Concern,Brake pad dust can be more toxic than exhaust ...,brake pad dust toxic exhaust emission study say,8
1,Environmental Benefit,EVs and datacentres driving new global ‘age of...,datacentres driving global age electricity say...,7
2,Economic Challenge,EV drivers spend extra £85m on VAT when using ...,driver spend extra vat using public charger,7
3,Economic Challenge,UK used electric vehicle sales hit record last...,uk used sale hit record last price fell,8
4,Environmental Benefit,Promoting green growth does not make you an ‘e...,promoting green growth econutter way forward,6


In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
from pyLDAvis import prepare
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer


os.environ["JOBLIB_MULTIPROCESSING"] = "0"

df = pd.read_csv("/content/TextMiningProject/data/cleaned_dataset.csv")
df = df.dropna(subset=['Cleaned_Description'])

MyCountVectorizer = CountVectorizer(max_features=100, max_df=0.7, min_df=5, stop_words='english')
X_counts = MyCountVectorizer.fit_transform(df['Cleaned_Description'])

vocab = MyCountVectorizer.get_feature_names_out()

lda = LatentDirichletAllocation(
    n_components=8,
    max_iter=10,
    random_state=42,
    learning_method='batch',
    n_jobs=1
)
lda.fit(X_counts)

topic_term_dists = lda.components_ / lda.components_.sum(axis=1)[:, np.newaxis]
doc_topic_dists = lda.transform(X_counts)
doc_topic_dists = doc_topic_dists / doc_topic_dists.sum(axis=1)[:, np.newaxis]
doc_lengths = np.asarray(X_counts.sum(axis=1)).flatten()
term_frequency = np.asarray(X_counts.sum(axis=0)).flatten()
vocab = MyCountVectorizer.get_feature_names_out()

data = {
    "topic_term_dists": topic_term_dists,
    "doc_topic_dists": doc_topic_dists,
    "doc_lengths": doc_lengths,
    "vocab": vocab,
    "term_frequency": term_frequency
}


pyLDAvis.enable_notebook()
vis_data = prepare(**data)
pyLDAvis.save_html(vis_data, "lda_byom_visualization.html")
vis_data

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.179677 -0.019099       1        1  23.175176
6      0.194190 -0.145628       2        1  13.323323
5     -0.391375 -0.249873       3        1  11.879298
2      0.158505  0.011199       4        1  11.646952
4     -0.137751  0.233739       5        1  11.591367
7     -0.141761  0.155979       6        1  10.202103
3      0.028715  0.185620       7        1   9.576994
0      0.109800 -0.171937       8        1   8.604788, topic_info=          Term        Freq       Total Category  logprob  loglift
12    charging  344.000000  344.000000  Default  30.0000  30.0000
19        cost  335.000000  335.000000  Default  29.0000  29.0000
26    emission  402.000000  402.000000  Default  28.0000  28.0000
61  newsletter  274.000000  274.000000  Default  27.0000  27.0000
21   dedicated  261.000000  261.000000  Default  26.0000  26.0000
..         ...         ...         ...      ...      ...      ...
38        grid    5.544892  149.168398   Topic8  -5.7031  -0.8393
74        sale    5.116858  163.630300   Topic8  -5.7835  -1.0122
30      future    9.750316  313.307367   Topic8  -5.1387  -1.0170
67     problem    4.694146  173.090047   Topic8  -5.8697  -1.1546
19        cost    4.919018  335.263253   Topic8  -5.8229  -1.7690

[292 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
0         1  0.293649  actually
0         2  0.315135  actually
0         4  0.179054  actually
0         5  0.071622  actually
0         7  0.128919  actually
...     ...       ...       ...
98        5  0.005422      work
98        7  0.097600      work
99        4  0.019519     world
99        5  0.951538     world
99        7  0.024398     world

[319 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 7, 6, 3, 5, 8, 4, 1])